In [1]:
# builtin modules
import re, json, sys
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

# external modules
import ampal
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform

# load local functions
sys.path.append('../src')
from data_gen_utils import *


# define the path to the colabfold results
colabfold_dir = Path('../../data/colabfold')

## Concatenate AF2 scores

Combine all a summary of all the AF2 scores into a single file. Takes ~ 2 minutes.

In [ ]:
# Get all JSON score files from the colabfold directory
score_paths = colabfold_dir.glob('*_scores_rank_*.json')

# Initialise an empty list to store score-related data
scores_lst = []

# Loop through each JSON score file path
for p in score_paths:
    # Extract the register (b, c, g) and rank number from the filename using regex
    reg, rank, seed = re.search('CC-Hex2-AB-3-([a-z]{1})_scores_rank_(\d{3})_.*_seed_(\d{3})', p.stem).groups()

    # Open and load the JSON file data
    with p.open('r') as json_obj:
        json_data = json.load(json_obj)
    
    # Calculate the summary scores for plDDT and pAE values from the loaded JSON data
    metric_scores = {}
    for metric in ['plddt', 'pae']:
        metric_scores[f'min_{metric}'] = np.min(json_data[metric])
        metric_scores[f'lq_{metric}'] = np.quantile(json_data[metric], 0.25)
        metric_scores[f'med_{metric}'] = np.quantile(json_data[metric], 0.5)
        metric_scores[f'mean_{metric}'] = np.mean(json_data[metric])
        metric_scores[f'std_{metric}'] = np.std(json_data[metric])
        metric_scores[f'uq_{metric}'] = np.quantile(json_data[metric], 0.75)
        metric_scores[f'max_{metric}'] = np.max(json_data[metric])

    # Extract pTM and ipTM scores from the JSON data
    ptm = json_data['ptm']
    iptm = json_data['iptm']

    # format the PDB path
    pdb_path = colabfold_dir / (p.stem.replace('scores', 'unrelaxed') + '.pdb')
    
    # Append the extracted and calculated values to the list as a dictionary
    scores_lst.append({
        'register': reg, 'pdb_path': pdb_path, 'score_path': p, 'rank': int(rank), 'seed': seed, \
        'ptm': ptm, 'iptm': iptm, **metric_scores
    })

# Convert the list of scores into a pandas DataFrame
scores_df = pd.DataFrame(scores_lst)

# sort the results
sorted_df = scores_df.sort_values(by=['register', 'rank'])


## Calculate Z-shfit

<img src="../assets/delta_z_example.png" alt="Zshift" width="200"/>

The Z-shift is calculated as the distance between the projections of the centroids of each helix (gray spheres) onto the super helical axis (red spheres).  For demonstration an antiparallel di-mer is shown (PDB ID: 7Q1T). Takes ~ 20 minutes.

In [6]:
# initialise emtpy columns to store the results
sorted_df['orientation'] = ''
sorted_df['dZ'] = np.nan

# Loop through each register ('b', 'c', 'g') and their corresponding structure paths
for reg in sorted_df.register.unique():

    # Enumerate through the PDB paths for each register
    for row_idx, row in sorted_df.query('register == @reg').iterrows():
        
        # Load the PDB file and select the first model (assuming multi-model PDBs)
        ampal_pdb = ampal.load_pdb(row['pdb_path'])[0]

        # Calculate the center of mass for the backbone of each chain in the structure
        coms = [ampal_pdb[chain.id].backbone.centre_of_mass for chain in ampal_pdb]

        # Initialize a list to store the Z-shift values between atoms
        dZ = []
        
        # Loop through pairs of center of mass coordinates to compute the Z-shift
        for i, atom1 in enumerate(coms):
            for atom2 in coms[i:]:
                # Calculate the Z-shift between atoms and append to the list
                dZ.append(calculate_z_shift_between_atoms(ampal_pdb, atom1, atom2))
                
        # Determine if the chains are parallel ('p') or anti-parallel ('ap')
        ori = p_or_ap(ampal_pdb)

        # Append the results for the current PDB, including region, id, max Z-shift, and orientation
        sorted_df.loc[row_idx, 'orientation'] = ori
        sorted_df.loc[row_idx, 'dZ'] = max(dZ)


In [7]:
# rearrange the columns
sorted_df = sorted_df[[
    'register', 'rank', 'orientation', 'seed', 'dZ',
    'min_plddt', 'lq_plddt', 'med_plddt', 'mean_plddt',
    'std_plddt','uq_plddt', 'max_plddt', 'min_pae', 'lq_pae',
    'med_pae', 'mean_pae','std_pae', 'uq_pae', 'max_pae',
    'ptm', 'iptm', 'pdb_path', 'score_path'
]]

# save the dataframe for future use
sorted_df.to_csv(colabfold_dir / 'af_scores_dZ.csv', index=False)

## TMScoring and clustering

<img src="../assets/tmscoring.png" alt="tmscoring" width="500"/>

In the paper, we cluster the high confidence (pTM > 0.7) AF2 predictions of the CC-Hex2-AB-3-***g*** to show the two structural classes of parallel and antiparallel helix orientations. This code enables the reproduction of our method. Takes ~ 15 minutes on 4 CPU cores.

In [14]:
# provide the path the US align programme
path_to_USalign = '/Users/jc17773/Downloads/USalign-master/USalign'

# read the file with scores
sorted_df = pd.read_csv(colabfold_dir / 'af_scores_dZ.csv')

# select only the confident g-register sequences
g_filtered_df = sorted_df.query('register == "g" and iptm > 0.7')

# get the PDB paths for filtered files
g_pdb_paths = g_filtered_df.pdb_path.values

# define a path to write results
g_tmscores_path = colabfold_dir / 'af_g-reg_TMscores.tsv'

# Prepare the output file 'g_alignment.tsv' and write the header for the alignment results
g_tmscores_path.write_text('PDBchain1\tPDBchain2\tTM1\tTM2\tRMSD\tID1\tID2\tIDali\tL1\tL2\tLali\n')

# Append alignment results to the output file, using parallel processing to speed up the task
with g_tmscores_path.open('a') as f:
    # Create a thread pool to execute multiple tasks concurrently (max 8 threads)
    with ThreadPoolExecutor(max_workers=4) as executor:
        # Submit alignment tasks to the thread pool for all pairs 
        # of structures in 'ranks_to_process'
        future_to_pair = {
            executor.submit(run_usalign, p1, p2, path_to_USalign): (p1, p2)
            for p1 in g_pdb_paths
            for p2 in g_pdb_paths
        }

        # As each task completes, write the result to the output file
        for future in as_completed(future_to_pair):
            result = future.result()  # Get the result of the alignment
            f.write(result + '\n')    # Append the result to the file

In [ ]:
# Load the alignment data from a TSV file and sort it by PDBchain1 and PDBchain2 columns
alignment_df = pd.read_csv(g_tmscores_path, sep='\t').sort_values(['PDBchain1', 'PDBchain2'])

# Calculate a normalized TM-score by adjusting the TM1 value based on the alignment length (Lali) and sequence length (L1)
alignment_df['TMnorm'] = alignment_df['TM1'] * (alignment_df['Lali'] / alignment_df['L1'])

# Extract the rank of each PDB chain from the 'PDBchain1' and 'PDBchain2' columns and convert to integer
alignment_df['rank1'] = alignment_df['PDBchain1'].str.extract('.*rank_(\d{3})').astype(int)
alignment_df['rank2'] = alignment_df['PDBchain2'].str.extract('.*rank_(\d{3})').astype(int)

# Create a pivot table where 'rank1' is the index, 'rank2' is the column, and the values are the normalized TM-scores (TMnorm)
tm_df = pd.pivot_table(data=alignment_df, index='rank1', columns='rank2', values='TMnorm')

# Convert the TM-score matrix into a TM distance matrix (TM-score is converted to distance by subtracting from 1)
tm_dist_arr = 1 - tm_df.values

# Ensure the distance matrix is symmetric by averaging it with its transpose
tm_dist_matrix = tm_dist_arr
tm_dist_matrix = (tm_dist_matrix + tm_dist_matrix.T) / 2

# Convert the symmetric TM distance matrix into a condensed distance matrix (needed for hierarchical clustering)
condensed_rmsd = squareform(tm_dist_matrix)

# Perform hierarchical clustering using the complete linkage method
Z = sch.linkage(condensed_rmsd, method='complete')

# Set the distance threshold to define clusters (TMscore is inverted now so distance of 0.05 = 0.95 TMscore)
distance_threshold = 0.05

# Assign cluster labels to each data point based on the distance threshold
cluster_labels = sch.fcluster(Z, distance_threshold, criterion='distance')

# Add the cluster labels to the dataframe, clusters are ordered by AF-M rank, as is the dataframe
g_filtered_df['cluster'] = cluster_labels

# As the dataframe is already sorted by AF-M rank, we can drop cluster duplciates and keep the 
# first entry to get best model scores for each cluster
display(g_filtered_df.drop_duplicates('cluster', keep='first'))

# Here we count the number of values in each cluster (1 is parallel, 2 is antiparallel)
display(g_filtered_df.value_counts('cluster'))

/var/folders/nr/f_s693214v54fjs9khpwyfnc0000gq/T/ipykernel_34488/2105797111.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_filtered_df['cluster'] = cluster_labels


,register,rank,orientation,seed,dZ,min_plddt,lq_plddt,med_plddt,mean_plddt,std_plddt,...,med_pae,mean_pae,std_pae,uq_pae,max_pae,ptm,iptm,pdb_path,score_path,cluster
3000,g,1,p,75,0.082253,84.56,94.015,96.560,95.216087,3.681121,...,1.97,2.795073,2.642873,2.70,17.78,0.90,0.90,../../data/colabfold/CCHex2AB-g_unrelaxed_rank...,../../data/colabfold/CCHex2AB-g_scores_rank_00...,1
3002,g,3,ap,3,5.557587,78.19,92.910,95.715,94.601667,4.186086,...,2.04,2.880784,2.824284,2.76,21.50,0.89,0.88,../../data/colabfold/CCHex2AB-g_unrelaxed_rank...,../../data/colabfold/CCHex2AB-g_scores_rank_00...,2


cluster
2    170
1      2
Name: count, dtype: int64